In [ ]:
# Import libraries
import ee
import folium
import geemap


In [ ]:
# Initialize the Earth Engine API
ee.Authenticate()
ee.Initialize(project = "class2-449500")


In [ ]:
# Load data
biodiversity_fc = ee.FeatureCollection("projects/class2-449500/assets/hotspots")
geom = ee.FeatureCollection("projects/e115-jgarciayi/assets/YanachagaChemillen").geometry().dissolve(maxError=1)

# Create categories: Tropical Andes vs Others
highlight_name = "Tropical Andes"
andes_fc = biodiversity_fc.filter(ee.Filter.eq('NAME', highlight_name))
others_fc = biodiversity_fc.filter(ee.Filter.neq('NAME', highlight_name))

# Style colors
color_andes = 'DDA0DDB3'   # Light purple with opacity
color_others = 'FFA500B3'  # Orange with opacity
color_border = '000000'    # Black border

# Style layers
andes_styled = andes_fc.style(color=color_andes[:6], fillColor=color_andes, width=1)
others_styled = others_fc.style(color=color_others[:6], fillColor=color_others, width=1)
boundary_fc = ee.FeatureCollection([ee.Feature(geom)]).style(color=color_border, fillColor='00000000', width=3)

# Get map center from protected area geometry
center = geom.centroid().coordinates().getInfo()[::-1]

# Create folium map
m = folium.Map(location=center, zoom_start=10)

# Add EE layers as TileLayers
andes_mapid = andes_styled.getMapId({})
others_mapid = others_styled.getMapId({})
boundary_mapid = boundary_fc.getMapId({})

folium.TileLayer(
    tiles=andes_mapid["tile_fetcher"].url_format,
    attr="EE",
    name="Tropical Andes",
    overlay=True,
    control=True
).add_to(m)

folium.TileLayer(
    tiles=others_mapid["tile_fetcher"].url_format,
    attr="EE",
    name="Other Hotspots",
    overlay=True,
    control=True
).add_to(m)

folium.TileLayer(
    tiles=boundary_mapid["tile_fetcher"].url_format,
    attr="EE",
    name="Protected Area Border",
    overlay=True,
    control=True
).add_to(m)

# Add working layer control
folium.LayerControl().add_to(m)

# Add working HTML legend (bottom-right)
legend_html = """
<div style="
  position: fixed;
  bottom: 30px;
  right: 10px;
  z-index: 9999;
  background-color: white;
  padding: 10px;
  border: 2px solid gray;
  font-size: 12px;
  box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
  max-height: 300px;
  overflow-y: auto;
">
<b>Legend</b><br>
<div><span style="background:#DDA0DD;width:12px;height:12px;display:inline-block;"></span> Tropical Andes Hotspot</div>
<div><span style="background:#FFA500;width:12px;height:12px;display:inline-block;"></span> Other Global Hotspots (zoom out to view)</div>
<div><span style="background:#000000;width:12px;height:12px;display:inline-block;"></span> Yanachaga-Chemillen National Park</div>
</div>
"""
legend = folium.Element(legend_html)
m.get_root().html.add_child(legend)

# Print the final interactive map
m


In [ ]:
# Save the interactive map to HTML
m.save("bird_biodiversity.html")